# PRMT-1181 Attachment Metadata Insights

### Context

PCSE want to be able to understand the scope and size of the GP2GP fallback service. They want to know more about the types of attachments that come via GP2GP.

They would like to know:
- Average and maximum file sizes
- Graph which demonstrates distribution of file sizes
- Identify File types that got as far as the transfer
- Add graph on number of attachments

### Assumptions

#### Each row of the attachments data set represents a unique attachment.

The core assumption made in this analysis is that each row is a different attachment. However, attachment ids do not uniquely idefity a each row of data within the same GP2GP transfer, despite sizes and types of attachments nevertheless varying. See appendix 1 for an example.

#### Attachments appear in the attachments dataset shortly after the transfer is requested.

This needs to be the case if we are to calculate how many gp2gp tranfers have no attachments.

This appears to be true (within two hours) for 99% of our data. See appendix 2 for more.

### Requirements

In order to replicate this notebook, perform the following steps:

1. Log into Splunk and run the following query, for 21/12/2020 00:00:00:00 to 27/12/2020 24:00:00 and 28/12/2020 00:00:00:00 to 03/01/2020 24:00:00 time frames (currently there are issues with downloading large data sets in Splunk):

```
index="spine2vfmmonitor" logReference=MPS0208
| fields _time, attachmentID, conversationID, FromSystem, ToSystem, attachmentType, Compressed, ContentType, LargeAttachment, Length, OriginalBase64
| fields - _raw
```

2. Download the two data sets as CSVs and place in a directory called `attachments_metadata`. Set the `INPUT_DATA_DIR` environment variable to point to the _parent_ of this directory.

3. Run the following Splunk query for the same time range, and place the CSV in a directory alongside the first one called `gp2gp_requests`.

```
index="spine2vfmmonitor" service="gp2gp" interactionID="urn:nhs:names:services:gp2gp/RCMR_IN010000UK05"
| fields _time, conversationID
| fields - _raw
```

Example directory layout, where `INPUT_DATA_DIR` is `/attachments`.
```
/attachments/attachments_metadata/attachments_one.csv
/attachments/attachments_metadata/attachments_two.csv
/attachments/gp2gp_requests/requests.csv
```


In [ ]:
import paths, os
import duckdb
from scripts.attachments import construct_attachments_db

In [ ]:
attachment_data_dir = os.environ["INPUT_DATA_DIR"]
cursor =  duckdb.connect()
construct_attachments_db(cursor, attachment_data_dir)
attachments = cursor.table("attachment_metadata")

## Attachment types

In [ ]:
(start_time, end_time) = attachments\
    .aggregate("MIN(time) as start_time, MAX(time) as end_time") \
    .execute().fetchone()

start_date = start_time.date()
end_date = end_time.date()

In [ ]:
attachment_count = attachments.aggregate("COUNT(*)").df()
print(f"{attachment_count.iat[0, 0]} attachments in dataset ({start_date} to {end_date})")

### Number of attachments by file type

In [ ]:
attachments_per_content_type = attachments.aggregate("content_type, count(*) as count").order("count DESC").df()
attachment_type_bars = attachments_per_content_type.plot.bar(
    x="content_type", y="count",
    title=f"Count of attachments by file type, {start_date} to {end_date}",
    rot=45,
    figsize=(16,8),
    legend=False,
)
attachment_type_bars.set(xlabel="Attachment file type", ylabel="Number of attachments")
attachment_type_bars.ticklabel_format(style='plain', axis='y')

In [ ]:
attachments_per_content_type.set_index("content_type").style.set_caption(f"Count of attachments by file type, {start_date} to {end_date}")

### Number of transfers with attachment

In [ ]:
transfer_count, with_attachment_count, without_attachment_count = cursor.execute("""
    select
        count(*) as transfers,
        sum(case when attachments.conversation_id is not null then 1 else 0 end) as with_attachments,
        sum(case when attachments.conversation_id is null then 1 else 0 end) as without_attachments
    from gp2gp_requests
    left join (select distinct conversation_id from attachment_metadata) attachments
    on attachments.conversation_id=gp2gp_requests.conversation_id
""").fetchone()
percent_with_attachment = with_attachment_count/transfer_count*100
print(f"Out of {transfer_count} transfers made between {start_date} and {end_date}, {with_attachment_count} had at least one attachment. ({percent_with_attachment}%) ")

### Number of attachments per transfer (excluding transfers with no attachments)

In [ ]:
attachments_per_transfer = attachments.aggregate("conversation_id, count(*) as count").df()

In [ ]:
attachments_per_transfer_truncated = attachments.aggregate("conversation_id, count(*) as count").filter("count <= 1000").df()

In [ ]:
attachments_per_transfer_hist_truncated = attachments_per_transfer_truncated.plot.hist(
    title=f"Histogram of attachments per transfer (where transfer has less than 1000 attachments), {start_date} to {end_date}",
    legend=False,
    bins=150, figsize=(14,8)
)
attachments_per_transfer_hist = attachments_per_transfer.plot.hist(
    title=f"Histogram of attachments per transfer, {start_date} to {end_date}",
    legend=False,
    bins=150, figsize=(14,8)
)
attachments_per_transfer_hist_truncated.set(xlabel="Attachments per transfer", ylabel="Frequency")
attachments_per_transfer_hist_truncated.ticklabel_format(style='plain', axis='y')
attachments_per_transfer_hist.set(xlabel="Attachments per transfer", ylabel="Frequency")
attachments_per_transfer_hist.ticklabel_format(style='plain', axis='y')

In [ ]:
attachments_per_transfer.describe().style.set_caption(f"Count of attachments per transfer, {start_date} to {end_date}")

## Size of attachments

In [ ]:
attachment_lengths = attachments.project("content_type, length/(1024.0*1024.0) as megabytes").df()

In [ ]:
attachment_lengths.describe().style.set_caption(f"Attachment file sizes (bytes), {start_date} to {end_date}")

In [ ]:
attachment_lengths_hist = attachment_lengths.plot.hist(
    title=f"Histogram of attachment size (logarithmic), {start_date} to {end_date}",
    legend=False,
    bins=100, log=True, figsize=(14,8)
)
attachment_lengths_hist.set(xlabel="Attachment size (MB)", ylabel="Frequency (logarithmic)")
attachment_lengths_hist.ticklabel_format(style='plain', axis='x', useOffset=False)

In [ ]:
attachment_lengths_hist_lin = attachment_lengths.plot.hist(
    title=f"Histogram of attachment size (linear), {start_date} to {end_date}",
    legend=False,
    bins=100, log=False, figsize=(14,8)
)
attachment_lengths_hist_lin.set(xlabel="Attachment size (MB)", ylabel="Frequency (linear)")
attachment_lengths_hist_lin.ticklabel_format(style='plain', axis='y', useOffset=False)

### Attachment size by file type

In [ ]:
attachment_lengths.groupby('content_type').describe().style.set_caption("Attachment file sizes by content type")


In [ ]:
attachment_lengths_boxplot = attachment_lengths.boxplot(
    'megabytes', by='content_type',
    figsize=(14,8),
    showfliers=False,
    rot=45,
)
attachment_lengths_boxplot.set_title(f"Boxplot of attachment size by content type (outliers removed), {start_date} to {end_date}")
attachment_lengths_boxplot.set(xlabel="Attachment type", ylabel="Size (MB)")

## Appendix 1: Attachment uniqueness

In [ ]:
attachments.aggregate("count(*)").df()

In [ ]:
cursor.execute("select count(*) from (select distinct conversation_id, attachment_id from attachment_metadata) uniq").df()

In [ ]:
cursor.execute("select count(*) from (select distinct conversation_id from attachment_metadata) uniq").df()

In [ ]:
converstaion_attachment_counts = attachments\
    .aggregate("conversation_id, count(*) as rows, count(distinct(attachment_id)) as unique_attachments")\
    .create_view("convo_attachment_counts")

In [ ]:
converstaion_attachment_counts.filter("rows != unique_attachments").df()

## Appendix 2: Combinding attachment logs and main spine logs

In [ ]:
gp2gp_attachments = cursor.execute("""
    select
        count(*) as row_count,
        sum(case when gp2gp_requests.conversation_id is null then 1 else 0 end) as attachments_with_no_transfer,
        sum(case when attachments.conversation_id is null then 1 else 0 end) as transfers_with_no_attachments
    from (select distinct conversation_id from attachment_metadata) attachments
    full outer join gp2gp_requests
    on attachments.conversation_id=gp2gp_requests.conversation_id
""").df()
gp2gp_attachments



In [ ]:
gp2gp_requests = cursor.execute("""
    select gp2gp_requests.time as req_time, attach.time as attach_time
    FROM gp2gp_requests
    LEFT JOIN (select conversation_id, max(time) as time from attachment_metadata group by conversation_id) attach
    ON attach.conversation_id=gp2gp_requests.conversation_id
""").df()

gp2gp_requests["time_delta"] = gp2gp_requests["attach_time"] - gp2gp_requests["req_time"]
gp2gp_requests["time_delta_sec"]= gp2gp_requests["time_delta"].astype('timedelta64[s]')
gp2gp_requests["time_delta_sec"].describe(percentiles=[.25, .50, .75, .90, .99]).apply(lambda x: format(x, '.1f'))